In [99]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import torch

In [65]:
import pandas as pd
import os
data_dir = '/data/vision/polina/scratch/barbaralam/cxrpe/data/Modeling'
df_neg = pd.read_csv(os.path.join(data_dir, 'EKGneg48.csv'))[:1675]
df_neg['label'] = 0
df_pos = pd.read_csv(os.path.join(data_dir, 'EKGpos48.csv'))
df_pos['label'] = 1
df = pd.concat([df_neg, df_pos])
# add path to ecg hea/dat
ecg_dir = '../data/ecgs/'
def ecg_path_fn(study_id):
    path = os.path.join(ecg_dir, str(int(study_id)))
    return path
df['ecg_path'] = df['study_id'].apply(ecg_path_fn)
df = df[df['ecg_path'].apply(lambda x: os.path.isfile(x+'.hea'))]
# remove one that cnanot be read by wfdb rdrecord
df = df[df['study_id'].apply(lambda x: int(x) not in [43010731])]
# reset index
df = df.reset_index(drop=True)

df

,note_id_x,subject_id,hadm_id_x,note_seq,charttime,storetime,text_x,Type_of_PE,study_id,file_name,...,note_id_1,subject_id_1,hadm_id_y,text_y,EKGtoCTA,time_elapsed_hours,within_24,and_48_hours,label,ecg_path
0,13294123-RR-174,13294123,24526753.0,174,8/20/43 15:56,8/20/43 17:20,EXAMINATION: CTA CHEST WITH CONTRAST\n\nINDIC...,NaN,47353099.0,47353099,...,13294123-EK-61,13294123.0,24526753.0,Clinical indication for EKG: Z79.899 - QT in...,1 days 00:32:00,24.533333,False,True,0,../data/ecgs/47353099
1,13294123-RR-174,13294123,24526753.0,174,8/20/43 15:56,8/20/43 17:20,EXAMINATION: CTA CHEST WITH CONTRAST\n\nINDIC...,NaN,43162156.0,43162156,...,13294123-EK-60,13294123.0,NaN,Clinical indication for EKG: R06.00 - Dyspne...,-1 days +20:18:00,-3.700000,True,True,0,../data/ecgs/43162156
2,13294123-RR-174,13294123,24526753.0,174,8/20/43 15:56,8/20/43 17:20,EXAMINATION: CTA CHEST WITH CONTRAST\n\nINDIC...,NaN,46257790.0,46257790,...,13294123-EK-63,13294123.0,24526753.0,Clinical indication for EKG: Z79.899 - QT in...,1 days 22:50:00,46.833333,False,True,0,../data/ecgs/46257790
3,13294123-RR-174,13294123,24526753.0,174,8/20/43 15:56,8/20/43 17:20,EXAMINATION: CTA CHEST WITH CONTRAST\n\nINDIC...,NaN,40626563.0,40626563,...,13294123-EK-62,13294123.0,24526753.0,Clinical indication for EKG: Z79.899 - QT in...,1 days 16:13:00,40.216667,False,True,0,../data/ecgs/40626563
4,15261136-RR-127,15261136,25709913.0,127,2/29/36 21:41,2/29/36 23:40,"HISTORY: Prior DVT/PE, presenting with sympto...",NaN,41316115.0,41316115,...,15261136-EK-28,15261136.0,25709913.0,Sinus rhythm. There are non-diagnostic Q wave...,0 days 04:27:00,4.450000,True,True,0,../data/ecgs/41316115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347,12750349-RR-11,12750349,26091418.0,11,2139-03-06 12:05:00,2139-03-06 14:34:00,CHEST CT PERFORMED ON ___.\n\nCOMPARISON: Non...,Acute,46483254.0,46483254,...,12750349-EK-14,12750349.0,NaN,Sinus rhythm. Variation in precordial lead pla...,-1 days +22:29:00,-1.516667,True,True,1,../data/ecgs/46483254
3348,14055709-RR-11,14055709,23063693.0,11,2137-06-29 21:31:00,2137-06-29 22:52:00,HISTORY: Concern for pulmonary embolus.\n\nTE...,Acute,49128807.0,49128807,...,14055709-EK-15,14055709.0,23063693.0,Prominent resting sinus tachycardia. Non-speci...,-1 days +18:55:00,-5.083333,True,True,1,../data/ecgs/49128807
3349,11343360-RR-37,11343360,20712282.0,37,2184-08-03 22:26:00,2184-08-04 09:46:00,HISTORY: ___ female with new onset hypoxia po...,Acute,48967203.0,48967203,...,11343360-EK-12,11343360.0,20712282.0,Sinus bradycardia. Left ventricular hypertrop...,-1 days +14:09:00,-9.850000,True,True,1,../data/ecgs/48967203
3350,11840035-RR-53,11840035,29098061.0,53,2147-08-12 09:37:00,2147-08-12 13:15:00,HISTORY: Sudden onset of hypoxia with elevate...,Subsegmental,49118027.0,49118027,...,11840035-EK-12,11840035.0,29098061.0,Atrial fibrillation with rapid ventricular res...,-1 days +19:09:00,-4.850000,True,True,1,../data/ecgs/49118027


In [66]:
paths = df['ecg_path'].tolist()

ecgs = []
for path in tqdm(paths):
    try:
        record = wfdb.rdrecord(path) 
        x = record.__dict__['p_signal']
    except:
        print(f'Cannot read {path}')
        x = None
    ecgs.append(x)
    

assert(all([x.shape==(5000,12) for x in ecgs]))

100%|██████████| 3352/3352 [00:12<00:00, 278.64it/s]


In [124]:
import argparse
# Arguments that will be saved in config file
parser = argparse.ArgumentParser(add_help=True,
                                 description='Train model to predict rage from the raw ecg tracing.')
parser.add_argument('--epochs', type=int, default=70,
                    help='maximum number of epochs (default: 70)')
parser.add_argument('--seed', type=int, default=2,
                    help='random seed for number generator (default: 2)')
parser.add_argument('--sample_freq', type=int, default=400,
                    help='sample frequency (in Hz) in which all traces will be resampled at (default: 400)')
parser.add_argument('--seq_length', type=int, default=4096,
                    help='size (in # of samples) for all traces. If needed traces will be zeropadded'
                                'to fit into the given size. (default: 4096)')
parser.add_argument('--scale_multiplier', type=int, default=10,
                    help='multiplicative factor used to rescale inputs.')
parser.add_argument('--batch_size', type=int, default=32,
                    help='batch size (default: 32).')
parser.add_argument('--lr', type=float, default=0.001,
                    help='learning rate (default: 0.001)')
parser.add_argument("--patience", type=int, default=7,
                    help='maximum number of epochs without reducing the learning rate (default: 7)')
parser.add_argument("--min_lr", type=float, default=1e-7,
                    help='minimum learning rate (default: 1e-7)')
parser.add_argument("--lr_factor", type=float, default=0.1,
                    help='reducing factor for the lr in a plateu (default: 0.1)')
parser.add_argument('--net_filter_size', type=int, nargs='+', default=[64, 128, 196, 256, 320],
                    help='filter size in resnet layers (default: [64, 128, 196, 256, 320]).')
parser.add_argument('--net_seq_lengh', type=int, nargs='+', default=[4096, 1024, 256, 64, 16],
                    help='number of samples per resnet layer (default: [4096, 1024, 256, 64, 16]).')
parser.add_argument('--dropout_rate', type=float, default=0.8,
                    help='dropout rate (default: 0.8).')
parser.add_argument('--kernel_size', type=int, default=17,
                    help='kernel size in convolutional layers (default: 17).')
parser.add_argument('--folder', default='model/',
                    help='output folder (default: ./out)')
parser.add_argument('--traces_dset', default='tracings',
                    help='traces dataset in the hdf5 file.')
parser.add_argument('--ids_dset', default='',
                    help='by default consider the ids are just the order')
parser.add_argument('--age_col', default='age',
                    help='column with the age in csv file.')
parser.add_argument('--ids_col', default=None,
                    help='column with the ids in csv file.')
parser.add_argument('--cuda', action='store_true',
                    help='use cuda for computations. (default: False)')
parser.add_argument('--n_valid', type=int, default=100,
                    help='the first `n_valid` exams in the hdf will be for validation.'
                         'The rest is for training')
parser.add_argument('path_to_traces',
                    help='path to file containing ECG traces')
parser.add_argument('path_to_csv',
                    help='path to csv file containing attributes.')

from rosemary import jpt_parse_args


cmd = """
--seq_length=4096 \
--folder=results/note_ecg_classification/resnet1d \
'1' \
'1' \
""".strip()

args = jpt_parse_args(parser, cmd=cmd)
args

Namespace(epochs=70, seed=2, sample_freq=400, seq_length=4096, scale_multiplier=10, batch_size=32, lr=0.001, patience=7, min_lr=1e-07, lr_factor=0.1, net_filter_size=[64, 128, 196, 256, 320], net_seq_lengh=[4096, 1024, 256, 64, 16], dropout_rate=0.8, kernel_size=17, folder='results/note_ecg_classification/resnet1d', traces_dset='tracings', ids_dset='', age_col='age', ids_col=None, cuda=False, n_valid=100, path_to_traces='1', path_to_csv='1')

In [125]:


torch.manual_seed(args.seed)
print(args)
# Set device
device = torch.device('cuda:0' if args.cuda else 'cpu')
folder = args.folder

# Generate output folder if needed
if not os.path.exists(args.folder):
    os.makedirs(args.folder)
# Save config file
with open(os.path.join(args.folder, 'args.json'), 'w') as f:
    json.dump(vars(args), f, indent='\t')


Namespace(epochs=70, seed=2, sample_freq=400, seq_length=4096, scale_multiplier=10, batch_size=32, lr=0.001, patience=7, min_lr=1e-07, lr_factor=0.1, net_filter_size=[64, 128, 196, 256, 320], net_seq_lengh=[4096, 1024, 256, 64, 16], dropout_rate=0.8, kernel_size=17, folder='results/note_ecg_classification/resnet1d', traces_dset='tracings', ids_dset='', age_col='age', ids_col=None, cuda=False, n_valid=100, path_to_traces='1', path_to_csv='1')


In [133]:
from sklearn.model_selection import train_test_split

X = np.stack([x.T for x in ecgs])
y = np.array(df['label'].tolist())
# take the middle portion of the signal
l = (X.shape[2] - args.seq_length)//2
r = l + args.seq_length
X = X[:,:,l:r]
print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(3352, 12, 4096) (3352,)
(2681, 12, 4096) (2681,) (671, 12, 4096) (671,)


In [175]:
from ecg_classification_utils import BatchDataloader
train_loader = BatchDataloader(X_train, y_train, bs=args.batch_size, mask=np.ones(len(X_train)).astype(bool))
valid_loader = BatchDataloader(X_test, y_test, bs=args.batch_size, mask=np.ones(len(X_test)).astype(bool))
print(len(train_loader), len(valid_loader))

84 21


In [176]:
from ecg_classification_utils import ResNet1d

tqdm.write("Define model...")
N_LEADS = 12  # the 12 leads
N_CLASSES = 1  # just the age
model = ResNet1d(input_dim=(N_LEADS, args.seq_length),
                 blocks_dim=list(zip(args.net_filter_size, args.net_seq_lengh)),
                 n_classes=N_CLASSES,
                 kernel_size=args.kernel_size,
                 dropout_rate=args.dropout_rate)
model.to(device=device)
tqdm.write("Done!")

tqdm.write("Define optimizer...")
optimizer = torch.optim.Adam(model.parameters(), args.lr)
tqdm.write("Done!")

tqdm.write("Define scheduler...")
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=args.patience,
                                                 min_lr=args.lr_factor * args.min_lr,
                                                 factor=args.lr_factor)
tqdm.write("Done!")

Epoch  0: train - Loss: 0.000000:   0%|          | 0/84 [01:04<?, ?it/s]

Define model...
Done!
Define optimizer...
Done!
Define scheduler...
Done!


In [185]:
loss_fn = torch.nn.BCEWithLogitsLoss()

def train(ep, dataload):
    model.train()
    total_loss = 0
    n_entries = 0
    train_desc = "Epoch {:2d}: train - Loss: {:.6f}"
    train_bar = tqdm(initial=0, leave=True, total=len(dataload),
                     desc=train_desc.format(ep, 0, 0), position=0)
    for X, y in dataload:
        X, y = X.to(device), y.to(device)
        # Reinitialize grad
        model.zero_grad()
        # Send to device
        # Forward pass
        y_pred = model(X)
        loss = loss_fn(y_pred, y.reshape(y_pred.shape))
        # Backward pass
        loss.backward()
        # Optimize
        optimizer.step()
        # Update
        bs = len(X)
        total_loss += loss.detach().cpu().numpy()
        n_entries += bs
        # Update train bar
        train_bar.desc = train_desc.format(ep, total_loss / n_entries)
        train_bar.update(1)
    train_bar.close()
    return total_loss / n_entries


def eval(ep, dataload):
    model.eval()
    total_loss = 0
    n_entries = 0
    eval_desc = "Epoch {:2d}: valid - Loss: {:.6f}"
    eval_bar = tqdm(initial=0, leave=True, total=len(dataload),
                    desc=eval_desc.format(ep, 0, 0), position=0)
    for X, y in dataload:
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            # Forward pass
            y_pred = model(X)
            loss = loss_fn(y_pred, y.reshape(y_pred.shape))
            # Update outputs
            bs = len(X)
            # Update ids
            total_loss += loss.detach().cpu().numpy()
            n_entries += bs
            # Print result
            eval_bar.desc = eval_desc.format(ep, total_loss / n_entries)
            eval_bar.update(1)
    eval_bar.close()
    return total_loss / n_entries

In [ ]:

tqdm.write("Training...")
start_epoch = 0
best_loss = np.Inf
history = pd.DataFrame(columns=['epoch', 'train_loss', 'valid_loss', 'lr',
                                'weighted_rmse', 'weighted_mae', 'rmse', 'mse'])
for ep in range(start_epoch, args.epochs):
    train_loss = train(ep, train_loader)
    valid_loss = eval(ep, valid_loader)
    # Save best model
    if valid_loss < best_loss:
        # Save model
        torch.save({'epoch': ep,
                    'model': model.state_dict(),
                    'valid_loss': valid_loss,
                    'optimizer': optimizer.state_dict()},
                   os.path.join(folder, 'model.pth'))
        # Update best validation loss
        best_loss = valid_loss
    # Get learning rate
    for param_group in optimizer.param_groups:
        learning_rate = param_group["lr"]
    # Interrupt for minimum learning rate
    if learning_rate < args.min_lr:
        break
    # Print message
    tqdm.write('Epoch {:2d}: \tTrain Loss {:.6f} ' \
              '\tValid Loss {:.6f} \tLearning Rate {:.7f}\t'
             .format(ep, train_loss, valid_loss, learning_rate))
    # Save history
    history = history.append({"epoch": ep, "train_loss": train_loss,
                              "valid_loss": valid_loss, "lr": learning_rate}, ignore_index=True)
    history.to_csv(os.path.join(folder, 'history.csv'), index=False)
    # Update learning rate
    scheduler.step(valid_loss)
tqdm.write("Done!")

Epoch  0: train - Loss: 0.000000:   0%|          | 0/84 [02:00<?, ?it/s]

Training...


Epoch  0: train - Loss: nan:  70%|███████   | 59/84 [01:18<00:32,  1.29s/it]    